In [1]:
%load_ext autoreload
%autoreload 2
from user_study_helpers import *

# Rekall Tutorial: Cyclist Detection

In this tutorial, you'll learn how to use Rekall and use it to detect a new class of object (cyclists) from existing person and bicycle detections.

Let's first load up some bounding box detections (from Mask R-CNN) and ground truth cyclist annotations:

In [2]:
bboxes_cydet = get_bboxes_cydet()
cyclist_gt = get_cyclist_bboxes()

We can use the `visualize_cydet` function to visualize them. Click on the video to expand it, and play the video by hovering over it and using `;`. You can navigate through the video by clicking through the timeline, and using the `+` and `-` buttons to zoom in or out.

In [3]:
visualize_cydet([bboxes_cydet, cyclist_gt])

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xcc\xfd\xcd\xae4K\x92\x9d\x07\xdf\x8a\xd0\xe3\x0f\…

# Filtering on Payload

Let's give a preview of some of the things you'll be able to do with Rekall. In the above two cells we've loaded up bounding box detections and cyclist annotations over two datasets, and visualized them for you.

Let's start by filtering the bounding box detections by class to look at bicycle and person detections:

In [4]:
bikes = bboxes_cydet.filter(lambda interval: interval['payload']['class'] == 'bicycle')

In [5]:
person = bboxes_cydet.filter(lambda interval: interval['payload']['class'] == 'person')

And now let's visualize them:

In [6]:
visualize_cydet([bikes, cyclist_gt, person])

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xcc\x9d\xcb\xae\xac\xc9q\x9d_E\xe0X \xf2~\xf1\xd0\…

In [ ]:
# Try some payload filtering functions yourself here!


# Filtering on Bounds

We are using Rekall's [Bounds3D](https://rekallpy.readthedocs.io/en/latest/source/rekall.bounds.html#rekall.bounds.Bounds3D) to represent these intervals. The intervals all have co-ordinates `t1`, `t2` (seconds), `x1`, `x2` (frame relative, between 0 and 1), and `y1`, `y2` (frame relative, between 0 and 1).

We can filter on the bounds co-ordinates as well:

In [7]:
visualize_cydet([
    bikes.filter(lambda interval: interval['t1'] < 300),
    cyclist_gt.filter(lambda interval: interval['t1'] < 300),
    person.filter(lambda interval: interval['t1'] < 300)
])

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xcc\x9d\xcb\xae\xac\xc9q\x9d_E\xe0X \xf2~\xf1\xd0\…

In [ ]:
# Try some bounds filtering functions yourself here!


# Creating new Intervals

Now that we have a flavor of what we can do with Rekall, let's build our understanding of the data representation from the ground up. Let's first understand what `Interval`s are - these are objects that we use to represent events in videos.

Here's a whiteboard drawing of what these Intervals can look like:
![videovolume_nested_sketch.jpg](https://olimar.stanford.edu/hdd/rekall_tutorials/basics/videovolume_nested_sketch.jpg)

We create new Intervals with a Bounds object, and an (optional) payload:

```Python
# This interval has time bounds from 0 to 10 seconds, X bounds from 0.5 to 0.7 (frame-relative),
# and Y bounds from 0.6 to 0.9 (frame-relative)
new_interval = Interval(Bounds3D(
    t1 = 0,
    t2 = 10,
    x1 = 0.5,
    x2 = 0.7,
    y1 = 0.6,
    y2 = 0.9
))

# This interval has time bounds from 5 to 15 seconds, and default X and Y bounds of the whole
# frame (0 to 1 for both X and Y)
new_interval2 = Interval(Bounds3D(5, 15))

# This interval has a payload. The payload can be an arbitrary object.
new_interval3 = Interval(Bounds3D(0, 1), payload={ 'class': 'my first payload' })
                         
# We can access the co-ordinates of payload an an Interval directly
print(new_interval['t1'], new_interval['t2'], new_interval['x1'])
print(new_interval2['t1'], new_interval2['x1'])
print(new_interval3['payload'])
print(new_interval3['payload']['class'])
```

Try it yourself below!

In [8]:
# This interval has time bounds from 0 to 10 seconds, X bounds from 0.5 to 0.7 (frame-relative),
# and Y bounds from 0.6 to 0.9 (frame-relative)
new_interval = Interval(Bounds3D(
    t1 = 0,
    t2 = 10,
    x1 = 0.5,
    x2 = 0.7,
    y1 = 0.6,
    y2 = 0.9
))

# This interval has time bounds from 5 to 15 seconds, and default X and Y bounds of the whole
# frame (0 to 1 for both X and Y)
new_interval2 = Interval(Bounds3D(5, 15))

# This interval has a payload. The payload can be an arbitrary object.
new_interval3 = Interval(Bounds3D(0, 1), payload={ 'class': 'my first payload' })

# We can access the co-ordinates of payload an an Interval directly
print(new_interval['t1'], new_interval['t2'], new_interval['x1'])
print(new_interval2['t1'], new_interval2['x1'])
print(new_interval3['payload'])
print(new_interval3['payload']['class'])

0 10 0.5
5 0.0
{'class': 'my first payload'}
my first payload


In [ ]:
# Create some Intervals yourself here!


# Associating Intervals with Events
In Rekall, we use *sets* of Intervals to represent events in videos. A single `IntervalSet` contains all occurrences of an event in a single video (all the bounding box detections, all the cyclist annotations, etc).

We can create an `IntervalSet` by passing in a list of `Interval`s:

```Python
# This IntervalSet represents all occurrences of a "made up" event in a video
my_first_intervalset = IntervalSet([
    Interval(Bounds3D(0, 10), payload = { 'class': 'made up'} ),
    Interval(Bounds3D(20, 30), payload = { 'class': 'made up'} ),
])
```

In [9]:
my_first_intervalset = IntervalSet([
    Interval(Bounds3D(0, 10), payload = { 'class': 'made up'} ),
    Interval(Bounds3D(20, 30), payload = { 'class': 'made up'} ),
])

One last thing - we want to associate each `IntervalSet` with the right video. We might have detected bikes in one video, but not the other!

We use `IntervalSetMapping` to associate `IntervalSet`s with different videos by keys. We create an `IntervalSetMapping` by passing in a `dict` from 

```Python
my_first_ism = IntervalSetMapping({
    0: IntervalSet(...), # the IntervalSet for video 0
    2: IntervalSet(...) # the IntervalSet for video 2
})
```

`bboxes_cydet` and `cyclist_gt` are `IntervalSetMapping` objects that we pre-loaded with `Interval`s containing object detections (`bboxes_cydet`) and ground-truth cyclist annotations (`cyclist_gt`).

Similarly, `bikes` and `person` are `IntervalSetMapping` objects representing the event that a bicycle object or a person object was detected in the video.

We can look at the keys of one of the `IntervalSetMapping` objects to see what the keys in our videos are!

In [10]:
bboxes_cydet.keys()

dict_keys([0, 3])

The videos that we were looking at before have keys 0 and 3. The visualization shows the videos in sorted order, so we know that the first video is video 0, and the second video is video 3.

Now we can create a simple `IntervalSetMapping` and visualize it:

In [11]:
my_first_ism = IntervalSetMapping({
    0: my_first_intervalset,
    3: IntervalSet([Interval(Bounds3D(50, 60, 0.5, 0.8, 0.1, 0.3)), Interval(Bounds3D(100,200))])
})

In [12]:
visualize_cydet([my_first_ism])

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xcdTMO\xc30\x0c\xfd+(g\xd4\xa6\xdd:`G\x8e\\\xe1\x8…

Now try it yourself below! Create some new `IntervalSetMapping` objects and visualize them on our videos.

NB: if you try to visualize an `IntervalSet` with only a single Interval in it, the timeline may not appear.

In [ ]:
# Try it yourself!


# Checkpoint: check on your understanding

At this point, you should be familiar with a few concepts:
* We represent events in videos as *sets* of intervals - objects that are defined by a Bounds object and an optional payload
* An `IntervalSet` represents all the occurrences of a particular event in a single video
* An `IntervalSetMapping` organizes multiple `IntervalSet`s and associates each one with a different video

Since we told you that `bboxes_cydet` and `cyclist_gt` are `IntervalSetMapping` objects, you may have also surmised that we have some functions for manipulating these sets of intervals - we've already looked at a few examples of the `filter` function. But now that we know more about the underlying data representation, we can take a closer look at `filter`.

From the [documentation](https://rekallpy.readthedocs.io/en/latest/index.html#rekall.IntervalSet.filter):

```Python
def filter(self, predicate):
    """
    Filter the set and keep intervals that pass the predicate.
    
    Args:
        predicate: A function that takes an Interval and returns a bool.
        
    Returns:
        A new IntervalSet which is the filtered set.
    """
```

So `filter` expects a function that will take an `Interval` and return `True` or `False`. It runs the predicate function on every interval, and only keeps the ones where the predicate returns `True`.

Notice that the documentation says that `filter` returns a new `IntervalSet` - that's because it's actually a function on `IntervalSet`! `IntervalSetMapping` simply reflects all the functions in `IntervalSet` and applies the functions to every `IntervalSet`. So even though we wrote the function over `IntervalSet`s, we can run them on `IntervalSetMapping` objects like `bboxes_cydet` and `cyclist_gt`.

## Checkpoint exercise: duplicate certain labels across the entirety of a video
Let's go through a simple exercise to check your understanding of these concepts.

We'll define a simple `IntervalSetMapping` with a few objects. We'll want you to duplicate the Intervals at time 0 of video 0 throughout videos 0 and 3, but only if the class in their payload is `car`.

In [13]:
objects_to_duplicate = IntervalSetMapping({
    0: IntervalSet([
        Interval(Bounds3D(0, 10, 0.1, 0.3, 0.1, 0.9), payload={ 'class': 'car' }),
        Interval(Bounds3D(0, 10, 0.4, 0.6, 0.1, 0.9), payload={ 'class': 'car' }),
        Interval(Bounds3D(0, 10, 0.7, 0.9, 0.1, 0.9), payload={ 'class': 'godzilla' }),
        Interval(Bounds3D(10, 20, 0.1, 0.3, 0.1, 0.9), payload={ 'class': 'car' }),
        Interval(Bounds3D(10, 20, 0.4, 0.6, 0.1, 0.9), payload={ 'class': 'car' }),
        Interval(Bounds3D(10, 20, 0.7, 0.9, 0.1, 0.9), payload={ 'class': 'godzilla' })
    ]),
    3: IntervalSet([
        Interval(Bounds3D(0, 10, 0.2, 0.4, 0.1, 0.9), payload={ 'class': 'car' }),
        Interval(Bounds3D(0, 10, 0.5, 0.7, 0.1, 0.9), payload={ 'class': 'car' }),
        Interval(Bounds3D(0, 10, 0.75, 0.95, 0.1, 0.9), payload={ 'class': 'godzilla' }),
        Interval(Bounds3D(10, 20, 0.2, 0.4, 0.1, 0.9), payload={ 'class': 'car' }),
        Interval(Bounds3D(10, 20, 0.5, 0.7, 0.1, 0.9), payload={ 'class': 'car' }),
        Interval(Bounds3D(10, 20, 0.75, 0.95, 0.1, 0.9), payload={ 'class': 'godzilla' })
    ])
})

In particular, we'll want the resulting Intervals to have the following properties:
* You should have Intervals with time extents of (0, 10), (10, 20), (20, 30), etc.
* The Intervals should have the same X/Y extent and payload as the Intervals at *time 0* of video 0 that have payload 'car'.
* The Intervals should cover the entire video.

A few hints to get you started:
* You can access `IntervalSet 0` of `objects_to_duplicate` like this: `objects_to_duplicate[0]`.
* You can loop through all the video keys of `objects_to_duplicate` using an iterator: `[k for k in objects_to_duplicate]`.
* You can filter down to Intervals that start at time 0 like this: `objects_to_duplicate.filter(lambda interval: interval['t0'] == 0)`
* You can access all the Intervals in an `IntervalSet` like this: `objects_to_duplicate[0].get_intervals()`. This will return a list of Intervals sorted by time.
* For example, to get the t2 value of the last interval in `bboxes_cydet`: `bboxes_cydet[0].get_intervals()[-1]['t2']`
* You can access the bounds of an interval like so: `interval['bounds']`
* You can copy a bounds like so: `interval['bounds'].copy()`
* Remember you can use Python ranges to get a value every ten seconds: `range(0, end, 10)`

In [14]:
# Construct an IntervalSetMapping with the answer here!
objects_at_start = objects_to_duplicate[0].filter(
    lambda interval: interval['t1'] == 0 and interval['payload']['class'] == 'car'
).get_intervals()

video_lengths = {
    key: bboxes_cydet[key].get_intervals()[-1]['t2']
    for key in bboxes_cydet
}

objects_duplicated = IntervalSetMapping({
    key: IntervalSet([
        Interval(Bounds3D(
            t1 = t,
            t2 = t + 10,
            x1 = interval['x1'],
            x2 = interval['x2'],
            y1 = interval['y1'],
            y2 = interval['y2']
        ))
        for t in range(0, int(video_lengths[key]), 10)
        for interval in objects_at_start
    ])
    for key in bboxes_cydet
})

visualize_cydet([objects_duplicated])

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xed\x9cMo\x9bh\x14\x85\xffJ\xc5zd\xf3\x9d&\xcbY\xc…

## Exercise solution:

Example solution to this exercise - don't look if you haven't tried it!

```Python
objects_at_start = objects_to_duplicate[0].filter(
    lambda interval: interval['t1'] == 0 and interval['payload']['class'] == 'car'
).get_intervals()

video_lengths = {
    key: bboxes_cydet[key].get_intervals()[-1]['t2']
    for key in bboxes_cydet
}

objects_duplicated = IntervalSetMapping({
    key: IntervalSet([
        Interval(Bounds3D(
            t1 = t,
            t2 = t + 10,
            x1 = interval['x1'],
            x2 = interval['x2'],
            y1 = interval['y1'],
            y2 = interval['y2']
        ))
        for t in range(0, int(video_lengths[key]), 10)
        for interval in objects_at_start
    ])
    for key in bboxes_cydet
})

visualize_cydet([objects_duplicated])
```

# Defining New Events through Composition and Manipulation Functions for Cyclist Detection

Now that we have that basic understanding of Rekall's data representation, let's look at some more complex manipulation functions to try to make a cyclist detector.

First, let's look at the [`join`](https://rekallpy.readthedocs.io/en/latest/index.html#rekall.IntervalSet.join) function. This function computes the cross product of two `IntervalSet`s, filters the resulting pairs by a predicate, and then merges the resulting pairs back into a single `Interval` using a merge operation.

Here's an example of using a `join` operation to create an `IntervalSetMapping` object containing instances of a `person` bounding box overlapping with a `bicycle` bounding box.
```Python
person_intersect_bike = person.join(
    bikes,
    predicate = and_pred(
        Bounds3D.T(equal()),
        Bounds3D.X(overlaps()),
        Bounds3D.Y(overlaps())
    ),
    merge_op = lambda interval1, interval2: Interval(
        interval1['bounds'].span(interval2['bounds'])
    ),
    window = 0.0,
    progress_bar = True
)
```

This function joins `person` and `bikes`. The predicate expects a function of the following format:
```Python
def predicate(interval1, interval2):
    # return True or False
```

We provide a number of spatial and temporal predicates in Rekall, outlined [here](https://rekallpy.readthedocs.io/en/latest/source/rekall.predicates.html). In this case, we're only keeping pairs of (person detection, bike detection) if they have the same time bounds (`Bounds3D.T(equal())`), and the `X` and `Y` bounds overlap (`Bounds3D.X(overlaps())`, `Bounds3D.Y(overlaps())`). The `and_pred` wrapper takes in an arbitrary number of predicates and makes sure that all of them pass.

The `merge_op` expects a function of this form:
```Python
def merge_op(interval1, interval2):
    # return a new Interval
```

In this case, we are returning a new Interval whose bounds span both the Intervals in the pair - basically, the minimum bounding box that covers both of them.

We pass in a `window` of `0.0` - this is an optimization that limits the pairs in the cross product to only those Intervals whose time bounds are apart from each other by `window` or less time. A `window` value of `0` limits the pairs to only those that overlap. Since we're already filtering by that in the time dimension, we know that this optimization won't change our results.

Finally, we pass `progress_bar=True` just to visualize a progress bar while we wait for this computation to complete (there are a lot of detections to process).

Let's run this function and visualize the results below!

In [15]:
# Use the join above to construct bounding boxes where a person and bike overlap!
person_intersect_bike = person.join(
    bikes,
    predicate = and_pred(
        Bounds3D.T(equal()),
        Bounds3D.X(overlaps()),
        Bounds3D.Y(overlaps())
    ),
    merge_op = lambda interval1, interval2: Interval(
        interval1['bounds'].span(interval2['bounds'])
    ),
    window = 0.0,
    progress_bar = True
)

100%|██████████| 2/2 [00:11<00:00,  5.77s/it]


Visualize the results: the top timeline shows evey time a person detection overlaps with a bicycle detection, and the bottom timeline shows the ground truth visualization again.

In [18]:
visualize_cydet([person_intersect_bike, cyclist_gt, bikes])

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xd4\x9d\xcb\xae\xa4\xc9u\x9d_\xc5\xe0\xd8 \xe2~\xf…

# Another join: the `minus` function

So looking for places where a person and bike bounding box overlap with each other seems to work pretty decently. But there are quite a few places where we're missing detections. Let's take a look at some of these cases.

We can use the [`minus`](https://rekallpy.readthedocs.io/en/latest/index.html#rekall.IntervalSet.minus) function to find all the instances where there is a ground truth annotation, but no detected "person intersecting a bike":

```Python
missed_detections = cyclist_gt.minus(
    person_intersect_bike,
    predicate = and_pred(
        Bounds3D.T(equal()),
        Bounds3D.X(overlaps()),
        Bounds3D.Y(overlaps()),
        iou_at_least(0.5)
    ),
    window = 0.0,
    progress_bar = True
)
```

This code takes any cyclist detection that overlaps with a `person_intersect_bike` Interval with [IOU](https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/) at least 0.5, and removes it from the set. The remaining Intervals are all the missing detections.

Try it yourself below (it'll take about a minute)!

In [19]:
# Use the minus function above to construct a set of missing detections!
missed_detections = cyclist_gt.minus(
    person_intersect_bike,
    predicate = and_pred(
        Bounds3D.T(equal()),
        Bounds3D.X(overlaps()),
        Bounds3D.Y(overlaps()),
        iou_at_least(0.5)
    ),
    window = 0.0,
    progress_bar = True
)

100%|██████████| 2/2 [01:00<00:00, 30.34s/it]


In [20]:
visualize_cydet([person_intersect_bike, missed_detections])

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xd4\x9d\xcb\xae\xa4\xc9u\x9d_\xc5\xe0\xd8 \xe2~\xf…

# Recursively merging Intervals with `coalesce`

If you've been following along, there should be a set of missed detections in the first video that we can fix. Zoom in on the first batch of detections for the first video. Notice that the we successfully detected cyclists on a few frames, but lost them again a few frames later! The root cause is that our object detector only sometimes detected the bicycle. But surely we should be able to do better - cyclists don't just pop in and out of existence!

Let's use the `coalesce` function along with some `map` functions to merge these detections together:

```Python
person_intersect_bike_coalesced = person_intersect_bike.dilate(
    15
).map(
    lambda interval: Interval(interval['bounds'], payload=[interval])
).coalesce(
    ('t1', 't2'),
    bounds_merge_op = lambda bounds1, bounds2: bounds1.span(bounds2),
    payload_merge_op = lambda p1, p2: p1 + p2,
    predicate = iou_at_least(0.5)
).dilate(-15)
```

Let's break down the above code:
* First, we `dilate` the time intervals so that any two cyclist detections that are less than 30 seconds away are now overlapping.
* Then, we use a `map` function to put the Intervals in their own payloads. Like `filter`, `map` takes a function that takes an Interval as input. The difference is that `map` outputs another Interval - it transforms all the intervals in an `IntervalSet`.
* Then, we `coalesce` the `IntervalSet`.
  * `coalesce` recursively merges all overlapping or touching Intervals along some axis (in this case, time).
  * We merge the bounds by taking the span of the existing Intervals and new Intervals, and merge the payloads by adding lists together.
  * We add in a predicate that we should only merge in new intervals if the IOU is at least 0.5.
  * At the end, we have a Interval that covers the entire spacetime volume that a cyclist occupied.
* Finally, we dilate by -15 seconds to undo the original dilation.

In [21]:
person_intersect_bike_coalesced = person_intersect_bike.dilate(
    15
).map(
    lambda interval: Interval(interval['bounds'], payload=[interval])
).coalesce(
    ('t1', 't2'),
    bounds_merge_op = lambda bounds1, bounds2: bounds1.span(bounds2),
    payload_merge_op = lambda p1, p2: p1 + p2,
    predicate = iou_at_least(0.5)
).dilate(-15)

In [22]:
visualize_cydet([person_intersect_bike_coalesced, cyclist_gt])

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xdc\xbd\xd9\xce$ir\x1d\xf8*B_7\x1a\xdf\xbe\xcc\xa5…

# Filtering a Coalesced Interval

Suppose we change the task, and say we now only want to detect the cyclists that appear for at least 30 seconds. We can use the `coalesce`'d versions of the cyclist detection to filter out cyclists that don't appear for long enough. We can also use the `split` function to turn the `IntervalSet`s back to representing individual cyclist detections, instead of large volumes that contain everywhere the cyclist was. Remember to un-dilate again!

```Python
person_intersect_bike_30_seconds = person_intersect_bike_coalesced.filter_size(
    min_size=30
).split(
    lambda interval: IntervalSet(interval['payload']).dilate(-15)
)
```

Try it below!

In [24]:
# Use filter_size and split above to get cyclists that appear on screen for at least 30 seconds...
person_intersect_bike_30_seconds = person_intersect_bike_coalesced.filter_size(
    min_size=30
).split(
    lambda interval: IntervalSet(interval['payload']).dilate(-15)
)

In [25]:
visualize_cydet([person_intersect_bike_30_seconds, cyclist_gt])

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xdc\xbd\xdb\xae%Yr\x1c\xf8+B?\x13\xc4\xba_\xe6Q\xb…